In [1]:
import tensorflow as tf
import numpy as np

C:\Users\anjineyulu\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<h3>Basic RNNs in TensorFlow</h3>

In [2]:
n_inputs=3
n_nuerons=5
Wx=tf.Variable(tf.random_normal(shape=[n_inputs,n_nuerons],dtype=tf.float32))
print(Wx)

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32_ref>


In [3]:
n_inputs=3
n_neurons=5
X0=tf.placeholder(tf.float32,[None,n_inputs])
X1=tf.placeholder(tf.float32,[None,n_inputs])

Wx=tf.Variable(tf.random_normal(shape=[n_inputs,n_nuerons],dtype=tf.float32))
Wy=tf.Variable(tf.random_normal(shape=[n_nuerons,n_nuerons],dtype=tf.float32))
b=tf.Variable(tf.zeros([1,n_neurons],dtype=tf.float32))

Y0=tf.tanh(tf.matmul(X0,Wx)+b)
Y1=tf.tanh(tf.matmul(Y0,Wy)+tf.matmul(X1,Wx)+b)

init=tf.global_variables_initializer()

In [4]:
X0_batch=np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])#t=0
X1_batch=np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])#t=1

with tf.Session() as sess:
    init.run()
    Y0_val,Y1_val=sess.run([Y0,Y1],feed_dict={X0:X0_batch,X1:X1_batch})

In [5]:
Y0_val

array([[ 0.7693296 ,  0.28638583,  0.4919593 ,  0.31841144, -0.5384732 ],
       [-0.7994223 ,  0.91202575,  0.9925498 ,  0.9998219 , -0.91308904],
       [-0.9967654 ,  0.9923967 ,  0.9999179 ,  0.99999994, -0.98633456],
       [-1.        ,  0.23324803,  0.09651235,  1.        ,  0.99999386]],
      dtype=float32)

In [6]:
Y1_val

array([[-0.9999989 ,  0.9996588 ,  0.99998134,  1.        , -0.9981369 ],
       [ 0.5960885 , -0.9651393 ,  0.5717339 ,  0.21870673, -0.9764459 ],
       [-0.99973136, -0.24565941,  0.9998588 ,  0.99999994, -0.99844056],
       [-0.99955076,  0.5537412 ,  0.97444844,  0.99980825, -0.4208229 ]],
      dtype=float32)

<h3>Static Unrolling Through Time</h3>

In [7]:
X0=tf.placeholder(tf.float32,[None,n_inputs])
X1=tf.placeholder(tf.float32,[None,n_inputs])

basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs,states=tf.contrib.rnn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32)

Y0,Y1=output_seqs


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [8]:
Y0

<tf.Tensor 'rnn/basic_rnn_cell/Tanh:0' shape=(?, 5) dtype=float32>

In [9]:
Y1

<tf.Tensor 'rnn/basic_rnn_cell/Tanh_1:0' shape=(?, 5) dtype=float32>

<h3>If there were 50 time steps</h3>

In [25]:
n_inputs=3
n_neurons=5
n_steps=2

<h3>Reset it if it is running multiple times</h3>

In [32]:
tf.reset_default_graph()

In [33]:
X=tf.placeholder(tf.float32,[None,n_steps,n_inputs]) 
X_seqs=tf.unstack(tf.transpose(X,perm=[1,0,2])) 
basic_cell=tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) 
output_seqs,states=tf.contrib.rnn.static_rnn(basic_cell,X_seqs,dtype=tf.float32) 
outputs=tf.transpose(tf.stack(output_seqs),perm=[1,0,2])

In [35]:
import numpy as np
X_batch=np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]],
])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val=outputs.eval(feed_dict={X:X_batch})

In [36]:
outputs_val

array([[[-0.32869458, -0.7165902 , -0.6505767 ,  0.53372616,
         -0.42741778],
        [-0.98965615, -0.5109421 , -1.        , -0.9944926 ,
         -0.9813475 ]],

       [[-0.8466361 , -0.86943793, -0.9985504 , -0.34975556,
         -0.90569335],
        [-0.09223882, -0.68356955, -0.33488235,  0.4809497 ,
         -0.26868585]],

       [[-0.9730642 , -0.9426082 , -0.9999951 , -0.8681875 ,
         -0.98786384],
        [-0.8825169 , -0.7547716 , -0.99992216, -0.97268236,
         -0.9777365 ]],

       [[ 0.76904196,  0.9418641 , -0.9999747 , -0.9629151 ,
         -0.9932602 ],
        [ 0.04130504, -0.22712985, -0.19511443, -0.8284035 ,
         -0.44059953]]], dtype=float32)

In [37]:
sess.graph

In [38]:
tf.get_default_graph()

In [39]:
for tensor in tf.get_default_graph().get_operations():
    print (tensor.name)

Placeholder
transpose/perm
transpose
unstack
rnn/Shape
rnn/strided_slice/stack
rnn/strided_slice/stack_1
rnn/strided_slice/stack_2
rnn/strided_slice
rnn/BasicRNNCellZeroState/ExpandDims/dim
rnn/BasicRNNCellZeroState/ExpandDims
rnn/BasicRNNCellZeroState/Const
rnn/BasicRNNCellZeroState/concat/axis
rnn/BasicRNNCellZeroState/concat
rnn/BasicRNNCellZeroState/zeros/Const
rnn/BasicRNNCellZeroState/zeros
rnn/BasicRNNCellZeroState/ExpandDims_1/dim
rnn/BasicRNNCellZeroState/ExpandDims_1
rnn/BasicRNNCellZeroState/Const_1
rnn/basic_rnn_cell/kernel/Initializer/random_uniform/shape
rnn/basic_rnn_cell/kernel/Initializer/random_uniform/min
rnn/basic_rnn_cell/kernel/Initializer/random_uniform/max
rnn/basic_rnn_cell/kernel/Initializer/random_uniform/RandomUniform
rnn/basic_rnn_cell/kernel/Initializer/random_uniform/sub
rnn/basic_rnn_cell/kernel/Initializer/random_uniform/mul
rnn/basic_rnn_cell/kernel/Initializer/random_uniform
rnn/basic_rnn_cell/kernel
rnn/basic_rnn_cell/kernel/Assign
rnn/basic_rnn_cell

In [40]:
g1=tf.Graph()

with g1.as_default():
    param_x=tf.placeholder(dtype=tf.float32,name='x')
    param_y=tf.placeholder(dtype=tf.float32,name='y')
    op_x_plus_y=tf.add(param_x,param_y,name='x_plus_y')

In [41]:
g1.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'y' type=Placeholder>,
 <tf.Operation 'x_plus_y' type=Add>]

In [42]:
tf.get_default_graph().get_operations()

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'transpose/perm' type=Const>,
 <tf.Operation 'transpose' type=Transpose>,
 <tf.Operation 'unstack' type=Unpack>,
 <tf.Operation 'rnn/Shape' type=Shape>,
 <tf.Operation 'rnn/strided_slice/stack' type=Const>,
 <tf.Operation 'rnn/strided_slice/stack_1' type=Const>,
 <tf.Operation 'rnn/strided_slice/stack_2' type=Const>,
 <tf.Operation 'rnn/strided_slice' type=StridedSlice>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/ExpandDims/dim' type=Const>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/ExpandDims' type=ExpandDims>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/Const' type=Const>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/concat/axis' type=Const>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/concat' type=ConcatV2>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/zeros/Const' type=Const>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/zeros' type=Fill>,
 <tf.Operation 'rnn/BasicRNNCellZeroState/ExpandDims_1/dim' type=Const>,
 <tf.Operation 'rnn/Ba

In [43]:
with tf.Session(graph=g1) as sess:
    result=sess.run(op_x_plus_y,feed_dict={param_x:20,param_y:1.9})

In [44]:
result

21.9

<h5>If you just want to evaluate a single operation, then instead</h5>

In [38]:
with tf.Session(graph=g1) as sess:
    op_x_plus_y.eval(feed_dict={param_x:20,param_y:1.9},session=sess)

In [39]:
op_x_plus_y

<tf.Tensor 'x_plus_y:0' shape=<unknown> dtype=float32>

<h5>The last argument can be skipped if you precede it with the following line of code using InteractiveSession()</h5>

In [40]:
tf.InteractiveSession(graph=g1)

In [42]:
op_x_plus_y.eval(feed_dict={param_x:20,param_y:1.9})

21.9

In [44]:
tf.InteractiveSession()
x=tf.constant(20.0)
y=tf.constant(1.1)
(x+y).eval()

C:\Users\anjineyulu\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


21.1

In [45]:
(x+1.1).eval()

21.1

As a procedural system TensorFlow also has mutable state which can be declared by tf.Variable()

n addition to placeholders and variables there are constants, defined by tf.constant().

In [48]:
graph=tf.Graph()
with graph.as_default():
    x=tf.Variable(tf.random_normal(shape=[]))

In [49]:
graph.get_operations()

[<tf.Operation 'random_normal/shape' type=Const>,
 <tf.Operation 'random_normal/mean' type=Const>,
 <tf.Operation 'random_normal/stddev' type=Const>,
 <tf.Operation 'random_normal/RandomStandardNormal' type=RandomStandardNormal>,
 <tf.Operation 'random_normal/mul' type=Mul>,
 <tf.Operation 'random_normal' type=Add>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>]

In [50]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(x))

0.43893775


In [51]:
x.dtype

tf.float32_ref

In [72]:
graph=tf.Graph()
with graph.as_default():
    x=tf.Variable(tf.zeros(dtype=tf.float32,shape=[]),trainable=True)
    c=tf.placeholder(dtype=tf.float32)
    f=(x-c)**2

In [73]:
graph.get_operations()

[<tf.Operation 'zeros' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>,
 <tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>]

In [74]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    print("x={}".format(sess.run(x)))
    print("f(x)={}".format(sess.run(f,feed_dict={c:3.5})))

x=0.0
f(x)=12.25


In [75]:
optim=tf.train.GradientDescentOptimizer(learning_rate=0.01)

In [76]:
with graph.as_default():
    grads_and_vars=optim.compute_gradients(f)
grads_and_vars

[(<tf.Tensor 'gradients/sub_grad/tuple/control_dependency:0' shape=() dtype=float32>,
  <tf.Variable 'Variable:0' shape=() dtype=float32_ref>)]

In [77]:
with graph.as_default():
    dfdx=grads_and_vars[0][0]
    print(len(grads_and_vars))
    print(len(grads_and_vars[0]))

1
2


In [78]:
dfdx

<tf.Tensor 'gradients/sub_grad/tuple/control_dependency:0' shape=() dtype=float32>

In [80]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    dfdx_0=sess.run(dfdx,feed_dict={c:3.5})
    print("df/dx ={}".format(dfdx_0))

df/dx =-7.0


In [81]:
with graph.as_default():
    train_step=optim.apply_gradients(grads_and_vars)

In [86]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    dfdx_0=sess.run(train_step,feed_dict={c:3.5})
    print("x = {}".format(sess.run(x)))

x = 0.07000000029802322


In [87]:
with graph.as_default():
    train_step=tf.train.GradientDescentOptimizer(0.025).minimize(f)

In [88]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        sess.run(train_step,feed_dict={c:3.5})
    print(sess.run(x))

3.4792776
